In [172]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import Dataset

In [373]:
xls_path = pd.ExcelFile('/Users/dileepsathyan/Documents/GitHub/Assignments/Vroom Virtusa/Analytics_Dataset.xlsx')
df_car_sales   = pd.read_excel(xls_path, 'car_sales')
df_car_details = pd.read_excel(xls_path, 'car_details')
df_car_claims  = pd.read_excel(xls_path, 'car_claims')

In [177]:
df_car_sales.head()

,car_id_sales,country,merchant_id,selling_date,selling_week,payment_date,sell_price,transport_to_merchant,transport_date
0,3421775349,ES,NaN,2021-04-29,2021-17,2021-05-14,1098,1,2021-05-31
1,5316743932,DE,NaN,2021-08-06,2021-31,2021-08-17,435,1,2021-08-31
2,5785159431,FR,NaN,2021-06-03,2021-22,2021-06-08,11028,0,NaN
3,7008763171,DE,NaN,2021-06-07,2021-23,2021-06-08,21100,1,NaN
4,1417558732,DE,NaN,2021-08-31,2021-35,2021-09-02,1943,1,2021-09-09


In [479]:
df_car_details.head()

,car_id_details,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
0,5096178328,1,1,0,1039,1141,1049,0,1100,3534.0
1,1879501444,0,1,1,1039,1138,1049,0,0,3534.0
2,8375508078,1,1,1,1039,1139,0,0,0,1606.0
3,1266330246,0,1,0,1040,1139,1050,0,0,3534.0
4,9079520882,0,0,0,1039,1141,0,0,1100,3534.0


In [480]:
df_car_claims.head()

,car_id,claim_number,full_refunds,partial_refunds,voucher_amount,claim_status,status_code
0,7525691980,NaN,0,0,NaN,NaN,0
1,8543168465,C1AG79687,0,1,620.140528,CLOSED_FULLY_PROCESSED,26
2,7308726237,NaN,0,0,NaN,NaN,0
3,9070724843,NaN,0,0,NaN,NaN,0
4,6274074693,NaN,0,0,NaN,NaN,0


# Data Cleaning

## 1. Car_Sales Dataframe Cleaning

In [481]:
df_car_sales.shape

(70512, 9)

In [482]:
df_car_sales.drop_duplicates(inplace=True)
df_car_sales.shape

(69618, 9)

#### Convert to right datatypes

In [483]:
# df_car_sales['car_id_sales'] = df_car_sales['car_id_sales'].astype('str')
# df_car_sales['merchant_id'] = df_car_sales['merchant_id'].astype('str')

df_car_sales['selling_date'] = pd.to_datetime(df_car_sales['selling_date'])
df_car_sales['payment_date'] = pd.to_datetime(df_car_sales['payment_date'])
df_car_sales['transport_date'] = pd.to_datetime(df_car_sales['transport_date'])

# Rename primary column
df_car_sales.rename(columns={'car_id_sales': 'car_id'}, inplace=True)

df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69618 entries, 0 to 70511
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   car_id                 69618 non-null  int64         
 1   country                69618 non-null  object        
 2   merchant_id            19088 non-null  float64       
 3   selling_date           69618 non-null  datetime64[ns]
 4   selling_week           69618 non-null  object        
 5   payment_date           69618 non-null  datetime64[ns]
 6   sell_price             69618 non-null  int64         
 7   transport_to_merchant  69618 non-null  int64         
 8   transport_date         32689 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(3), object(2)
memory usage: 5.3+ MB


In [484]:
# Notice that the same car_ids are repeated for different merchant_ids

df_car_sales[df_car_sales['car_id'] == 5805619905]

,car_id,country,merchant_id,selling_date,selling_week,payment_date,sell_price,transport_to_merchant,transport_date
24388,5805619905,BE,16191229.0,2021-07-31,2021-30,2021-08-03,22596,1,2021-08-04
46739,5805619905,BE,14307587.0,2021-07-31,2021-30,2021-08-03,22596,1,2021-08-04


In [485]:
# There are merchant_ids who bought so many cars.

df_car_sales.merchant_id.value_counts()

8819318.0     823
5482442.0      97
2593085.0      92
6717821.0      64
2491964.0      62
             ... 
3451474.0       1
6902952.0       1
5854417.0       1
6903110.0       1
10485917.0      1
Name: merchant_id, Length: 9102, dtype: int64

In [486]:
#### As long as we have no clarity on the correct Merchant who bought a car, it doesnt matter which record we delete.
#### Lets keep car_id only once and delete all following records for the sale.

df_car_sales = df_car_sales.drop_duplicates(subset='car_id', keep="first")

# Reset index
df_car_sales.reset_index(drop=True, inplace=True)

df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69551 entries, 0 to 69550
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   car_id                 69551 non-null  int64         
 1   country                69551 non-null  object        
 2   merchant_id            19021 non-null  float64       
 3   selling_date           69551 non-null  datetime64[ns]
 4   selling_week           69551 non-null  object        
 5   payment_date           69551 non-null  datetime64[ns]
 6   sell_price             69551 non-null  int64         
 7   transport_to_merchant  69551 non-null  int64         
 8   transport_date         32654 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(3), object(2)
memory usage: 4.8+ MB


In [487]:
df_car_sales.head()

,car_id,country,merchant_id,selling_date,selling_week,payment_date,sell_price,transport_to_merchant,transport_date
0,3421775349,ES,NaN,2021-04-29,2021-17,2021-05-14,1098,1,2021-05-31
1,5316743932,DE,NaN,2021-08-06,2021-31,2021-08-17,435,1,2021-08-31
2,5785159431,FR,NaN,2021-06-03,2021-22,2021-06-08,11028,0,NaT
3,7008763171,DE,NaN,2021-06-07,2021-23,2021-06-08,21100,1,NaT
4,1417558732,DE,NaN,2021-08-31,2021-35,2021-09-02,1943,1,2021-09-09


## Car_Details Dataframe Cleaning

In [488]:
df_car_details.shape

(70512, 10)

In [489]:
df_car_details.drop_duplicates(inplace=True)
df_car_details.shape

(70034, 10)

In [490]:
df_car_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70034 entries, 0 to 70511
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   car_id_details     70034 non-null  int64  
 1   has_tuning         70034 non-null  int64  
 2   has_airbags        70034 non-null  int64  
 3   has_alarm_system   70034 non-null  int64  
 4   fuel_type          70034 non-null  int64  
 5   gear_type          70034 non-null  int64  
 6   ac_type            70034 non-null  int64  
 7   xenon_light        70034 non-null  int64  
 8   navigation_system  70034 non-null  int64  
 9   radio_system       43494 non-null  float64
dtypes: float64(1), int64(9)
memory usage: 5.9 MB


#### Convert to right datatypes

In [491]:
# df_car_details['car_id_details'] = df_car_details['car_id_details'].astype('str')

# Rename primary column
df_car_details.rename(columns={'car_id_details': 'car_id'}, inplace=True)
df_car_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70034 entries, 0 to 70511
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   car_id             70034 non-null  int64  
 1   has_tuning         70034 non-null  int64  
 2   has_airbags        70034 non-null  int64  
 3   has_alarm_system   70034 non-null  int64  
 4   fuel_type          70034 non-null  int64  
 5   gear_type          70034 non-null  int64  
 6   ac_type            70034 non-null  int64  
 7   xenon_light        70034 non-null  int64  
 8   navigation_system  70034 non-null  int64  
 9   radio_system       43494 non-null  float64
dtypes: float64(1), int64(9)
memory usage: 5.9 MB


In [492]:
## Check for duplicate records of same car_id

df_car_details[df_car_details.groupby('car_id')['car_id'].transform('size') > 1].sort_values(by='car_id').head(20)

,car_id,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
49297,1021854212,0,0,0,1039,1141,0,0,1100,3534.0
25281,1021854212,0,0,0,1040,1141,0,0,1100,3534.0
19591,1071192762,1,1,1,1040,1139,1050,1089,1100,3534.0
37616,1071192762,1,1,1,1039,1139,1050,1089,1100,3534.0
20260,1092285954,1,1,0,1039,1138,1050,0,1100,NaN
62249,1092285954,1,1,0,1039,1138,1050,0,0,NaN
23369,1116277714,1,1,0,1039,1138,0,0,0,3534.0
62436,1116277714,1,1,0,1040,1138,0,0,1100,3534.0
50603,1130791238,1,1,0,1039,1141,1050,0,1100,1100.0
12934,1130791238,1,1,0,1040,1141,1050,0,1100,1100.0


In [493]:
# Subset the records with duplicate car_ids

df_cd_dup = df_car_details[df_car_details.groupby('car_id')['car_id'].transform('size') > 1].sort_values(by='car_id')
df_cd_dup.sort_values(by='car_id').head(10)

,car_id,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
49297,1021854212,0,0,0,1039,1141,0,0,1100,3534.0
25281,1021854212,0,0,0,1040,1141,0,0,1100,3534.0
19591,1071192762,1,1,1,1040,1139,1050,1089,1100,3534.0
37616,1071192762,1,1,1,1039,1139,1050,1089,1100,3534.0
20260,1092285954,1,1,0,1039,1138,1050,0,1100,NaN
62249,1092285954,1,1,0,1039,1138,1050,0,0,NaN
23369,1116277714,1,1,0,1039,1138,0,0,0,3534.0
62436,1116277714,1,1,0,1040,1138,0,0,1100,3534.0
50603,1130791238,1,1,0,1039,1141,1050,0,1100,1100.0
12934,1130791238,1,1,0,1040,1141,1050,0,1100,1100.0


In [494]:
df_cd_dup.shape

(961, 10)

In [495]:
# Remove the duplicated car_ids from original dataframe to make it a deduplicated df.

df_cd_deduped = df_car_details.drop(df_cd_dup.index)
df_cd_deduped.shape

(69073, 10)

##### Notice from the above subset that fule_type and navigation_system are the 2 fields which cause duplication.

###### 1. Those who differ only by fuel_type, lets keep the first record of each car_id and delete the rest.

In [496]:
df_cd_dup = df_cd_dup.drop_duplicates(['car_id', 'navigation_system']).sort_values(by='car_id')

df_cd_dup.head(10)

,car_id,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
49297,1021854212,0,0,0,1039,1141,0,0,1100,3534.0
19591,1071192762,1,1,1,1040,1139,1050,1089,1100,3534.0
20260,1092285954,1,1,0,1039,1138,1050,0,1100,NaN
62249,1092285954,1,1,0,1039,1138,1050,0,0,NaN
23369,1116277714,1,1,0,1039,1138,0,0,0,3534.0
62436,1116277714,1,1,0,1040,1138,0,0,1100,3534.0
50603,1130791238,1,1,0,1039,1141,1050,0,1100,1100.0
11699,1131021388,1,1,0,1039,1138,1050,0,0,3534.0
24110,1134904596,0,0,0,1040,1141,0,0,0,3534.0
48854,1175003748,1,1,0,1040,1138,1050,0,1100,1100.0


In [497]:
df_cd_dup.shape

(792, 10)

In [498]:
# Filter the uniques and append to the deduped dataframe & drop the uniques from the df_cd_dup

df_temp = df_cd_dup[df_cd_dup.groupby('car_id')['car_id'].transform('size') == 1]

df_cd_deduped = df_cd_deduped.append(df_temp)
df_cd_dup = df_cd_dup.drop(df_temp.index)
df_cd_dup.head(10)

,car_id,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
20260,1092285954,1,1,0,1039,1138,1050,0,1100,NaN
62249,1092285954,1,1,0,1039,1138,1050,0,0,NaN
23369,1116277714,1,1,0,1039,1138,0,0,0,3534.0
62436,1116277714,1,1,0,1040,1138,0,0,1100,3534.0
54450,1189899240,0,0,0,1039,1141,0,0,0,3534.0
11135,1189899240,0,0,0,1040,1141,0,0,1100,3534.0
52138,1251496465,1,1,1,1039,1139,1050,1089,0,3534.0
67790,1251496465,1,1,1,1039,1139,1050,1089,1100,3534.0
8106,1257596002,0,0,0,1039,1141,0,0,0,3534.0
67243,1257596002,0,0,0,1039,1141,0,0,1100,3534.0


In [499]:
df_cd_dup.shape

(628, 10)

##### 2. Those who differ only by navigation_system, lets keep the first record of each car_id and delete the rest.

In [500]:
df_cd_dup = df_cd_dup.drop_duplicates(['car_id', 'fuel_type']).sort_values(by='car_id')

df_cd_dup.head(10)

,car_id,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
20260,1092285954,1,1,0,1039,1138,1050,0,1100,NaN
23369,1116277714,1,1,0,1039,1138,0,0,0,3534.0
62436,1116277714,1,1,0,1040,1138,0,0,1100,3534.0
54450,1189899240,0,0,0,1039,1141,0,0,0,3534.0
11135,1189899240,0,0,0,1040,1141,0,0,1100,3534.0
52138,1251496465,1,1,1,1039,1139,1050,1089,0,3534.0
8106,1257596002,0,0,0,1039,1141,0,0,0,3534.0
69446,1257782013,1,1,0,1039,1138,0,0,0,NaN
29230,1257782013,1,1,0,1040,1138,0,0,1100,NaN
54861,1266182032,0,0,0,1039,1141,0,0,1100,3534.0


In [501]:
# Filter the uniques and append to the deduped dataframe & drop the uniques from the df_cd_dup

df_temp = df_cd_dup[df_cd_dup.groupby('car_id')['car_id'].transform('size') == 1]

df_cd_deduped = df_cd_deduped.append(df_temp)
df_cd_dup = df_cd_dup.drop(df_temp.index)
df_cd_dup.head(10)

,car_id,has_tuning,has_airbags,has_alarm_system,fuel_type,gear_type,ac_type,xenon_light,navigation_system,radio_system
23369,1116277714,1,1,0,1039,1138,0,0,0,3534.0
62436,1116277714,1,1,0,1040,1138,0,0,1100,3534.0
54450,1189899240,0,0,0,1039,1141,0,0,0,3534.0
11135,1189899240,0,0,0,1040,1141,0,0,1100,3534.0
69446,1257782013,1,1,0,1039,1138,0,0,0,NaN
29230,1257782013,1,1,0,1040,1138,0,0,1100,NaN
64274,1550589125,0,0,0,1039,1141,0,0,0,3534.0
2198,1550589125,0,0,0,1040,1141,0,0,1100,3534.0
47017,1675413187,0,0,0,1039,1141,0,0,0,3534.0
47934,1675413187,0,0,0,1040,1141,0,0,1100,3534.0


In [502]:
df_cd_dup.shape

(134, 10)

###### 3. Those who differ in both navigation_system & fuel_type, lets keep the record with 1100 as nav_system and delete the 0 values.

In [503]:
df_cd_dup = df_cd_dup[df_cd_dup['navigation_system'] != 0]
df_cd_deduped = df_cd_deduped.append(df_cd_dup)
df_cd_deduped.shape

(69551, 10)

In [504]:
df_car_details = df_cd_deduped

# Reset index
df_car_details.reset_index(drop=True, inplace=True)

df_car_details.shape

(69551, 10)

## Car_Claims Dataframe Cleaning

In [374]:
df_car_claims.shape

(70512, 6)

In [375]:
df_car_claims.drop_duplicates(inplace=True)
df_car_claims.shape

(70504, 6)

In [376]:
df_car_claims.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70504 entries, 0 to 70511
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   car_id_claims    70504 non-null  int64  
 1   claim_number     19974 non-null  object 
 2   full_refunds     70504 non-null  int64  
 3   partial_refunds  70504 non-null  int64  
 4   voucher_amount   15173 non-null  float64
 5   claim_status     19974 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 3.8+ MB


#### Convert to right datatypes

In [377]:
# df_car_claims['car_id_claims'] = df_car_claims['car_id_claims'].astype('str')
# df_car_claims['claim_number'] = df_car_claims['claim_number'].astype('str')

# Rename primary column
df_car_claims.rename(columns={'car_id_claims': 'car_id'}, inplace=True)

df_car_claims.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70504 entries, 0 to 70511
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   car_id           70504 non-null  int64  
 1   claim_number     19974 non-null  object 
 2   full_refunds     70504 non-null  int64  
 3   partial_refunds  70504 non-null  int64  
 4   voucher_amount   15173 non-null  float64
 5   claim_status     19974 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 3.8+ MB


In [378]:
df_car_claims['claim_status'].value_counts()

CLOSED_FULLY_PROCESSED                                                    7910
REJECTED                                                                  5026
CLOSED_NO_COMPENSATION                                                    3410
CREATED                                                                   1262
CLOSED_ITEMS_FOUND_AND_SENT                                                783
CLOSED_WAITING_FOR_MORE_PROOF                                              348
PROOF_NOT_RECEIVED                                                         309
WAITING_FOR_SALES_TO_NEGOTIATE_WITH_THE_MERCHANT                           234
WAITING_FOR_LOCAL_OPS_MD_DECISION                                          159
MISSING_ITEMS_IN_PROGRESS                                                  134
2ND_DECISION_WAITING_FOR_SALES_TO_NEGOTIATE_WITH_MERCHANT                   77
NEW                                                                         72
WAITING_FOR_CLAIMS_TEAM_2_EVALUATION                

In [379]:
status_codes = { np.NaN: 0,
                'CREATED': 1,
                'NEW_MISSING_ITEM_CLAIM': 1,
                'NEW': 1,

                'PROOF_NOT_RECEIVED': 2,
                'CLOSED_WAITING_FOR_MORE_PROOF': 3,
                'CLOSED_OVER_TIME_LIMIT': 4,
                'MORE_PROOF_RECEIVED': 5,
                'WAITING_FOR_CLAIMS_TEAM_1_EVALUATION': 6,
                '2ND_EVALUATION_CLAIMS_TEAM_2': 7,
                'WAITING_FOR_CLAIMS_TEAM_2_EVALUATION': 8,
                'WAITING_FOR_CARRIER_FEEDBACK': 9,
                'WAITING_FOR_LEGAL_TEAM': 10,

                'CLOSED_NO_COMPENSATION': 11,
                'REJECTED': 12,

                'MISSING_ITEMS_IN_PROGRESS': 13,
                'CLOSED_ITEMS_FOUND_AND_SENT': 14,

                'WAITING_FOR_SALES_TO_NEGOTIATE_WITH_THE_MERCHANT': 15,
                '2ND_DECISION_WAITING_FOR_SALES_TO_NEGOTIATE_WITH_MERCHANT': 16,
                'WAITING_FOR_LOCAL_OPS_MD_DECISION': 17,
                '2ND_DECISION_WAITING_FOR_LOCAL_OPS_MD_DECISION': 18,

                'CLOSED_CARRIER_WILL_COMPENSATE': 19,

                'CLOSED_WAITING_FOR_MERCHANT_BANK_DETAILS': 20, 
                'CLOSED_WAITING_FOR_CAR_DOCUMENTS_AND_BANK_DETAILS': 21,
                'WAITING_FOR_BANK_DETAILS_VALIDATION': 22,

                'CLOSED_WAITING_FOR_MONEY_TO_BE_SHIPPED_AND_COMPENSATION_TO_BE_RECORDED': 23,
                'CLOSED_WAITING_FOR_COMPENSATION_TO_BE_RECORDED': 24,
                'CLOSED_WAITING_FOR_DOCUMENTS': 25,
                'CLOSED_FULLY_PROCESSED': 26 }

In [423]:
df_cc = df_car_claims.copy()
df_cc['status_code'] = df_cc['claim_status'].map(status_codes)
# df_cc = df_cc.sort_values(by=['car_id', 'status_code'])
df_cc.head()

,car_id,claim_number,full_refunds,partial_refunds,voucher_amount,claim_status,status_code
0,7525691980,NaN,0,0,NaN,NaN,0
1,8543168465,C1AG79687,0,1,620.140528,CLOSED_FULLY_PROCESSED,26
2,7308726237,NaN,0,0,NaN,NaN,0
3,9070724843,NaN,0,0,NaN,NaN,0
4,6274074693,NaN,0,0,NaN,NaN,0


In [424]:
# df_cc.groupby(['car_id'])['status_code'].max()

idx = df_cc.groupby(['car_id'])['status_code'].transform(max) == df_cc['status_code']
df_cc = df_cc[idx]
df_cc.shape

(69640, 7)

In [425]:
# Identifying Pending Duplicates in the df

df_cc[df_cc['car_id']== 5149303881]

,car_id,claim_number,full_refunds,partial_refunds,voucher_amount,claim_status,status_code
10787,5149303881,C4TK46454,0,1,3112.880206,CLOSED_FULLY_PROCESSED,26
19814,5149303881,C4TK46454,1,0,NaN,CLOSED_FULLY_PROCESSED,26


In [426]:
df_cc['car_id'].nunique()

69551

In [429]:
# Subset the records with duplicate car_ids

df_cc_dup = df_cc[df_cc.groupby('car_id')['car_id'].transform('size') > 1].sort_values(by='car_id')
df_cc_dup.head(10)

,car_id,claim_number,full_refunds,partial_refunds,voucher_amount,claim_status,status_code
65138,1091480557,C1ZJ20861,0,1,3826.101648,CLOSED_FULLY_PROCESSED,26
4190,1091480557,C1ZJ20861,1,0,NaN,CLOSED_FULLY_PROCESSED,26
37116,1266182032,C1SS06287,0,1,2804.535297,CLOSED_FULLY_PROCESSED,26
6935,1266182032,C1SS06287,0,1,2149.457901,CLOSED_FULLY_PROCESSED,26
29497,1329241519,C2GR87979,0,1,705.036285,CLOSED_FULLY_PROCESSED,26
44485,1329241519,C2GR87979,0,1,482.114221,CLOSED_FULLY_PROCESSED,26
42278,1356214392,C1KB80609,0,1,2797.550353,REJECTED,12
25037,1356214392,C1KB80609,0,1,2337.572390,REJECTED,12
68046,1370035853,C2KX69437,0,1,1074.665772,CLOSED_FULLY_PROCESSED,26
31235,1370035853,C2KX69437,0,1,1394.864155,CLOSED_FULLY_PROCESSED,26


In [430]:
df_cc_dup.shape

(177, 7)

In [449]:
# Remove the duplicated car_ids from original dataframe to make it a deduplicated df.

df_cc_deduped = df_cc.drop(df_cc_dup.index)
df_cc_deduped.shape

(69463, 7)

In [432]:
df_cc_dup['car_id'].nunique()

88

In [450]:
df_cc_deduped['car_id'].nunique()

69463

##### Noticed that there are duplicate records with same claim_status
##### 1. Lets keep the record with voucher_amount for the duplicated records.

In [434]:
df_cd_dup.shape

(67, 10)

In [435]:
# 1a. Before deleting the rows with NULL voucher_amount, 
#     make sure that no duplicated car_ids have NULL values in both of the rows

df_cc_dup[df_cc_dup['voucher_amount'].isnull()]['car_id'].value_counts()

2802433468    2
9320910782    1
2818110547    1
2612408606    1
3855907037    1
7363990747    1
5036405350    1
4191587287    1
7151950304    1
8813163856    1
4427773773    1
3539094478    1
3842127670    1
4922195103    1
5149303881    1
7348849733    1
4065361055    1
4309735904    1
6974366625    1
4566417827    1
7613645214    1
5570835365    1
3096043603    1
5239366824    1
5755284393    1
4197008170    1
7096519625    1
1951678892    1
1091480557    1
9565824175    1
3157164086    1
1993742522    1
4998841339    1
9336612599    1
Name: car_id, dtype: int64

In [436]:
df_cc_dup[df_cc_dup['car_id']== 2802433468]

,car_id,claim_number,full_refunds,partial_refunds,voucher_amount,claim_status,status_code
60052,2802433468,C1CJ23334,0,0,NaN,CREATED,1
10175,2802433468,C1CJ23334,0,0,NaN,NEW,1


In [452]:
# Since there exists 1 record which has NULL values in both of its rows, 
# lets first clean it to take 1 row and append to the deduped dataframe.

df_cc_deduped = df_cc_deduped.append(df_cc_dup.loc[[60052]])
df_cc_deduped.shape

(69464, 7)

In [455]:
# Now drop the both rows of the car_id from dup dataframe.

df_cc_dup = df_cc_dup.drop([60052, 10175])
df_cc_dup.shape

(175, 7)

In [458]:
df_cc_dup.head()

,car_id,claim_number,full_refunds,partial_refunds,voucher_amount,claim_status,status_code
65138,1091480557,C1ZJ20861,0,1,3826.101648,CLOSED_FULLY_PROCESSED,26
4190,1091480557,C1ZJ20861,1,0,NaN,CLOSED_FULLY_PROCESSED,26
37116,1266182032,C1SS06287,0,1,2804.535297,CLOSED_FULLY_PROCESSED,26
6935,1266182032,C1SS06287,0,1,2149.457901,CLOSED_FULLY_PROCESSED,26
29497,1329241519,C2GR87979,0,1,705.036285,CLOSED_FULLY_PROCESSED,26


In [464]:
# 1b. Now go on and delete the rows with NULL values in voucher_amount in the duplicated dataframe.

idx = df_cc_dup[df_cc_dup['voucher_amount'].isnull()].index

df_cc_dup = df_cc_dup.drop(idx)
df_cc_dup.shape

(142, 7)

In [469]:
# Append the unique rows to dedupted dataframe

df_temp = df_cc_dup[df_cc_dup.groupby('car_id')['car_id'].transform('size') == 1]
df_cc_deduped = df_cc_deduped.append(df_temp)

# Drop the deduped df from the duplicate dataframe
df_cc_dup = df_cc_dup.drop(df_temp.index)
df_cc_dup.shape

(69497, 7)

In [473]:
df_cc_dup.head()

,car_id,claim_number,full_refunds,partial_refunds,voucher_amount,claim_status,status_code
37116,1266182032,C1SS06287,0,1,2804.535297,CLOSED_FULLY_PROCESSED,26
6935,1266182032,C1SS06287,0,1,2149.457901,CLOSED_FULLY_PROCESSED,26
29497,1329241519,C2GR87979,0,1,705.036285,CLOSED_FULLY_PROCESSED,26
44485,1329241519,C2GR87979,0,1,482.114221,CLOSED_FULLY_PROCESSED,26
42278,1356214392,C1KB80609,0,1,2797.550353,REJECTED,12


In [474]:
# 2. Now voucher_amount field contains only valid amounts. Lets keep the rows with max amounts and drop the rest.

idx = df_cc_dup.groupby(['car_id'])['voucher_amount'].transform(max) == df_cc_dup['voucher_amount']
df_cc_dup = df_cc_dup[idx]
df_cc_dup.shape

(54, 7)

In [477]:
# Append all the deduped rows to the 

df_cc_deduped = df_cc_deduped.append(df_cc_dup)
df_car_claims = df_cc_deduped

# Reset index
df_car_claims.reset_index(drop=True, inplace=True)

df_car_claims.shape

(69551, 7)

# Combine the dataframes to 1

In [505]:
df_car_sales.shape

(69551, 9)

In [506]:
df_car_details.shape

(69551, 10)

In [507]:
df_car_claims.shape

(69551, 7)

In [508]:
df_merged = pd.merge(df_car_claims, df_car_details, how='left', on='car_id')
# df_merged = pd.merge(df_merged, df_car_sales, how='inner', on='car_id')
df_merged.shape

(69551, 16)

In [509]:
df_merged = pd.merge(df_merged, df_car_sales, how='inner', on='car_id')
df_merged.shape

(69551, 24)

In [511]:
df_merged.car_id.nunique()

# merchant_id is different which causes duplicate entries

69551